# Groundedness Checking Plugins

적절한 구성 설정 및 설정을 위해서는 [첫 번째](./00-getting-started.ipynb) 시작하기 노트북의 시작 부분에 설명된 단계를 따르세요.

대규모 언어 모델(LLM)의 잘 알려진 문제점은 사물을 만들어낸다는 것입니다. 이를 '환각'이라고 부르기도 하지만 더 안전한(그리고 덜 의인화된) 용어는 '근거 없는 추가'로, 텍스트에서 확고하게 확립할 수 없는 것을 말합니다. LLM 응답의 내용이 '사실'인지 여부를 확인하려는 경우 제공된 프롬프트에서 이를 확인하거나('좁은 근거'라고 함) 일반적인 지식을 사용할 수 있습니다('넓은 근거'). 좁은 근거를 사용하면 '사실이지만 근거가 없는 것'으로 분류될 수 있다는 점에 유의하세요. 예를 들어 “나는 스위스에 산다”는 사실임에 틀림없지만 “나는 제네바에 산다”는 _협소하게_ 근거가 없습니다(_광범위하게_ 근거가 있음).

이 노트북에서는 간단한 근거 파이프라인을 실행해 요약 텍스트에 원본과 비교해 근거가 없는 추가 사항이 있는지 확인하고, 이 정보를 사용해 요약 텍스트를 개선합니다. 이 작업은 세 단계로 진행됩니다:

1. 요약 텍스트에 있는 엔티티 목록 만들기
1. 이러한 엔티티가 원본(근거) 텍스트에 나타나는지 확인합니다.
1. 요약 텍스트에서 근거가 없는 엔티티를 제거합니다.

이 맥락에서 '엔티티'란 무엇인가요? 가장 간단한 형태로는 사람이나 장소(예: '딘' 또는 '시애틀')와 같은 명명된 객체입니다. 하지만 개념과 관련된 _주장_일 수도 있습니다(예: '딘은 시애틀 근처에 산다'). 이 노트북에서는 명명된 객체의 더 간단한 경우에만 집중하겠습니다.


Initial configuration for the notebook to run properly.

In [1]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

Let us define our grounding text:

In [2]:
grounding_text = """저는 태어날 때부터 제네바 사람이고, 제 가족은 그 공화국에서 가장 저명한 가문 중 하나입니다.
제 조상들은 오랜 세월 동안 카운슬러와 신디케이터로 활동했고, 제 아버지는 여러 공식적인 자리에서
명예와 명성을 얻었죠. 그는 청렴하고 지칠 줄 모르는 관심으로 그를 아는 모든 사람들로부터 존경을 받았습니다.
존경받으셨죠. 아버지는 젊은 시절을 국가 업무에 몰두하며 보냈습니다.
여러 가지 상황으로 인해 일찍 결혼하지 못했고, 인생의 쇠퇴기에 이르러서야 남편이 되었고
한 가정의 아버지가 되었습니다.

그의 결혼 생활은 그의 성격을 잘 보여주기 때문에 언급하지 않을 수 없습니다. 그의
가장 친밀한 친구 중 한 명은 번영을 누리다가 수많은 불운을 겪으며 가난에 빠진 상인이었습니다.
보퍼트라는 이름의 이 사람은 교만하고 거침없는 성품으로 가난하게 사는 걸 견디지 못했습니다.
가난과 망각 속에서 사는 것을 견디지 못했습니다. 과거에는 높은 지위와 위엄으로 유명했던 나라에서 말이죠. 따라서
가장 명예로운 방식으로 빚을 갚은 그는 딸과 함께 루체른 마을로 후퇴했습니다,
그곳에서 그는 무명의 비참한 삶을 살았습니다. 아버지는 보포트를 진정한 우정으로 사랑했고, 그가 이런 곳에서 후퇴하는 것을
이 불행한 상황에 처한 그의 후퇴를 깊이 슬퍼했습니다. 아버지는 친구의 잘못된 자존심 때문에
두 사람을 하나로 묶어준 애정에 걸맞지 않은 행동을 한 친구를 비통해했습니다. 그는 그를 찾으려는 노력에 시간을 낭비하지 않았습니다,
그의 신용과 도움을 통해 세상을 다시 시작하도록 설득하기 위해서였습니다.

보퍼트는 자신을 감추기 위해 효과적인 조치를 취했고, 아버지가 그의 거처를 발견하기까지 10개월이 걸렸습니다.
거처를 발견하기까지 10개월이 걸렸습니다. 이 발견에 기뻐하며 아버지는 서둘러 로이스 강변에 있는 그 집으로 향했습니다.
하지만 집에 들어서자 비참함과 절망만이 그를 맞이했습니다. 보퍼트는 재산의 파탄으로 인해
그의 재산의 난파, 그러나 몇 달 동안 생계를 유지하기에 충분했고 그 동안 그는
그는 상인의 집에서 존경할만한 일자리를 얻기를 희망했습니다. 결과적으로 그 간격은 다음과 같이 보냈습니다.
그의 슬픔은 사색할 여유가 생겼을 때 더욱 깊어지고 고조되었다.
그의 마음을 너무 빨리 붙잡아 세 달이 끝날 무렵 그는 병상에 누워 어떤 노력도 할 수 없었습니다.

그의 딸은 가장 큰 부드러움으로 그를 돌 보았지만 그녀는 절망으로 그들의 작은 기금이
급속히 줄어들고 있고 다른 지원의 전망도 없다는 것을 절망적으로 보았습니다. 그러나 캐롤라인 보퍼트는 흔치 않은 마음의 소유자였습니다.
용기를 내어 역경 속에서 그녀를 지탱해 주었습니다. 그녀는 평범한 일을 구하고 짚을 엮고
그리고 여러 가지 방법을 동원해 겨우 생활에 필요한 약간의 돈을 벌기 위해 노력했습니다.

이런 식으로 몇 달이 지났습니다. 그녀의 아버지는 점점 더 나빠졌고, 그녀의 시간은 온통 아버지를 돌보는 데만 쓰였다;
그녀의 생계 수단은 감소했습니다. 그리고 열 번째 달에 그녀의 아버지는 그녀의 품에서 죽었고 그녀는 고아가되었고
거지. 이 마지막 타격은 그녀를 극복했고, 그녀는 아버지가 들어 왔을 때 보 포터의 관 옆에 무릎을 꿇고 몹시 울었습니다.
방에 들어갔다. 그는 자신을 돌보는 데 헌신 한 불쌍한 소녀를 보호하는 정신처럼 왔습니다.
친구의 장례식이 끝난 후 그는 그녀를 제네바로 데려가 친척의 보호 아래 두었습니다. 2년
이 사건 이후 캐롤라인은 그의 아내가 되었습니다."""

We will load our settings and get the LLM service to use for the notebook.

In [3]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [4]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )

## Import the Plugins

We are going to be using the grounding plugin, to check its quality, and remove ungrounded additions:


In [6]:
# note: using plugins from the samples folder
plugins_directory = "../prompt_template_samples/"

groundingSemanticFunctions = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="GroundingPlugin")

We can also extract the individual semantic functions for our use:


In [7]:
entity_extraction = groundingSemanticFunctions["ExtractEntities"]
reference_check = groundingSemanticFunctions["ReferenceCheckEntities"]
entity_excision = groundingSemanticFunctions["ExciseEntities"]

## Calling Individual Semantic Functions

We will start by calling the individual grounding functions in turn, to show their use. For this we need to create a same summary text:


In [8]:
summary_text = """
밀라노의 존경받는 거주자였던 아버지는 보퍼트라는 상인의 절친한 친구였는데, 일련의 불행 끝에
가난하게 취리히로 이주했습니다. 아버지는 친구의 어려움에 화가 나 그를 찾아갔습니다,
비열한 거리에서 그를 발견했죠. 보포트는 소액의 돈을 모았지만 그와 그의 딸 메리를 부양하기에는 충분하지 않았습니다.
그의 딸 메리. 메리는 생계를 유지하기 위해 일을 구했지만 10개월 만에 아버지가 돌아가시고
그녀는 거지가되었습니다. 아버지가 그녀를 도와주었고 2년 후 두 사람은 로마를 방문해 결혼했습니다.
"""

summary_text = summary_text.replace("\n", " ").replace("  ", " ")
print(summary_text)

 밀라노의 존경받는 거주자였던 아버지는 보퍼트라는 상인의 절친한 친구였는데, 일련의 불행 끝에 가난하게 취리히로 이주했습니다. 아버지는 친구의 어려움에 화가 나 그를 찾아갔습니다, 비열한 거리에서 그를 발견했죠. 보포트는 소액의 돈을 모았지만 그와 그의 딸 메리를 부양하기에는 충분하지 않았습니다. 그의 딸 메리. 메리는 생계를 유지하기 위해 일을 구했지만 10개월 만에 아버지가 돌아가시고 그녀는 거지가되었습니다. 아버지가 그녀를 도와주었고 2년 후 두 사람은 로마를 방문해 결혼했습니다. 


Some things to note:

- The implied residence of Geneva has been changed to Milan
- Lucerne has been changed to Zurich
- Caroline has been renamed as Mary
- A reference to Rome has been added

The grounding plugin has three stages:

1. Extract entities from a summary text
2. Perform a reference check against the grounding text
3. Excise any entities which failed the reference check from the summary

Now, let us start calling individual semantic functions.


### Extracting the Entities

The first function we need is entity extraction. We are going to take our summary text, and get a list of entities found within it. For this we use `entity_extraction()`:


In [9]:
extraction_result = await kernel.invoke(
    entity_extraction,
    input=summary_text,
    topic="people and places",
    example_entities="존, 제인, 어머니, 형제, 파리, 로마",
)

print(extraction_result)

<entities>
- 밀라노 (Milan, a place)
- 아버지 (Father, a person)
- 보퍼트 (Beaufort, a person)
- 취리히 (Zurich, a place)
- 메리 (Mary, a person)
- 로마 (Rome, a place)
</entities>


So we have our list of entities in the summary


### Performing the reference check

We now use the grounding text to see if the entities we found are grounded. We start by adding the grounding text to our context:


With this in place, we can run the reference checking function. This will use both the entity list in the input, and the `reference_context` in the context object itself:


In [10]:
grounding_result = await kernel.invoke(reference_check, input=extraction_result.value, reference_context=grounding_text)

print(grounding_result)

<ungrounded_entities>
- 밀라노 (Milan, a place)
- 취리히 (Zurich, a place)
- 메리 (Mary, a person)
- 로마 (Rome, a place)
</ungrounded_entities>


So we now have a list of ungrounded entities (of course, this list may not be well grounded itself). Let us store this in the context:


### Excising the ungrounded entities

Finally we can remove the ungrounded entities from the summary text:


In [11]:
excision_result = await kernel.invoke(entity_excision, input=summary_text, ungrounded_entities=grounding_result.value)

print(excision_result)

<context>
아버지는 보퍼트라는 상인의 절친한 친구였는데, 일련의 불행 끝에 가난하게 이주했습니다. 아버지는 친구의 어려움에 화가 나 그를 찾아갔습니다, 비열한 거리에서 그를 발견했죠. 보포트는 소액의 돈을 모았지만 그와 그의 딸을 부양하기에는 충분하지 않았습니다. 그의 딸. 그녀는 생계를 유지하기 위해 일을 구했지만 10개월 만에 아버지가 돌아가시고 그녀는 거지가되었습니다. 아버지가 그녀를 도와주었고 2년 후 두 사람은 방문해 결혼했습니다.
</context>
